In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests
import re
from dateutil import parser
from datetime import datetime

# Scrape Auto.bg


In [2]:
models = []
prices = []
years = []
kilometers = []
transmissions = []
engine_types = []
horsepowers = []

In [3]:
def scrape_current_page_Auto(): 
    for result in results:
        try:
            model = result.find("div", class_ = "link").find("a").text
            models.append(model.strip())
        except:
            models.append(np.nan)

        try:
            price = result.find("div", class_ = "price").text
            prices.append(price)
        except:
            prices.append(np.nan)

        try:
            year = result.find("div", class_ = "info").text.split('|')[0].strip('\n').strip()
            years.append(year)
        except:
            years.append(np.nan)

        try:
            km = result.find("div", class_ = "info").text.split('|')[1].strip('\n').strip().split()[0]
            kilometers.append(km)
        except:
            kilometers.append(np.nan)

        try:
            engine_type = result.find("div", class_ = "info").text.split('|')[3].strip('\n').strip()
            engine_types.append(engine_type.strip())
        except:
            engine_types.append(np.nan)

        try:
            horsepower = result.find("div", class_ = "info").text.split('к.с.')[0].split('|')[-1].strip(' \n')
            horsepowers.append(horsepower)
        except:
            horsepowers.append(np.nan)

In [4]:
strings = ['abarth', 'acura', 'alpina', 'audi', 
       'aston-martin','austin','alfa-romeo', 'bentley',
       'bertone', 'bugatti'
       'bmw', 'buick', 'cadillac',
       'chevrolet','chrysler','citroen', 
       'corvette', 'cupra', 'dacia','daewoo',
       'daihatsu','dodge', 
       'ds', 'dr', 'eagle',
        'ferrari','fiat', 'ford', 'fso', 'gaz',
       'gmc', 'gonow', 'great-wall',
       'haval', 'honda',
        'hyundai','hummer', 'ifa', 'infiniti','innocenti', 
       'isuzu','iveco', 'jaguar', 'jeep','kia',
        'lada','laforza','lamborghini','lancia',
       'land-rover','lotus',
        'lexus', 'lincoln', 'mahindra', 'maybach'
        'maserati', 'mazda','mclaren', 'mg',
        'mercedes-benz', 'mini',
        'mitsubishi', 'moskvich','oldsmobile',
        'nissan', 'opel', 
        'peugeot', 'porsche',
        'pontiac',
        'renault', 'rolls-royce',
        'rover', 'saab', 
        'seat', 'skoda',
        'ssang-yong','ssangyong','subaru','suzuki',
        'smart','tata','tazzari',
        'tesla', 'toyota', 'uaz',
        'trabant','volga','volvo', 'vw']

In [5]:
for string in strings:
    url = 'https://www.auto.bg/obiavi/avtomobili-dzhipove/' + string + '/page/1'
    count = 1
    while True:
        # Get URL  
        response = requests.get(url)
        soup = BeautifulSoup(response.text, 'html.parser')
        # Results
        results = soup.findAll("div", class_ = "resultItem")
        if len(results) == 0:
            break
        scrape_current_page_Auto()
        count = count + 1
        next_url = 'https://www.auto.bg/obiavi/avtomobili-dzhipove/' + string + '/page/' + str(count)
        url = next_url

In [6]:
df_auto = pd.DataFrame({'Model' : models,'Price': prices,'Year': years, 
                               'Kilometers' : kilometers,'Engine_type': engine_types,
                                'Horsepower': horsepowers})

In [7]:
df_auto.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 39283 entries, 0 to 39282
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Model        39283 non-null  object
 1   Price        39283 non-null  object
 2   Year         39283 non-null  object
 3   Kilometers   39283 non-null  object
 4   Engine_type  39283 non-null  object
 5   Horsepower   39283 non-null  object
dtypes: object(6)
memory usage: 1.8+ MB


In [8]:
df_auto.head(10)

,Model,Price,Year,Kilometers,Engine_type,Horsepower
0,Abarth 124,250 000 лв.,февруари 2020 г.,11111,Бензин,"регион Варна, гр. Варна11:26 часа от днес"
1,Abarth 595 595 PISTA 19000km,39 999 лв.,февруари 2020 г.,19000,Бензин,165
2,Abarth 595,45 800 лв.,юни 2019 г.,54350,Бензин,181
3,Abarth 595 Pista,36 000 лв.,септември 2017 г.,45000,Бензин,165
4,Abarth 595,21 800 лв.,април 2014 г.,80000,Бензин,165
5,Abarth 595,25 700 лв.,май 2015 г.,86000,Бензин,160
6,Abarth 595 695 tributo ferrari,56 500 лв.,септември 2010 г.,39400,Бензин,180
7,Abarth 595 TURISMO*500*PANORAMA,51 000 лв.,август 2021 г.,8200,Бензин,165
8,Abarth 595 500 PISTA 595 CABRIO NAVI EVRO 6B 4...,40 900 лв.,декември 2017 г.,48000,Бензин,160
9,Acura Mdx 3.7-БЕНЗИН-АВТОМАТ,24 442 лв.,април 2009 г.,160543,Бензин,301


# Data Cleaning

In [9]:
d = {'EUR': 1.95, 'USD': 1.90, 'лв.': 1}
s1 = df_auto['Price'].replace(d, regex=True)
s2 = df_auto['Price'].str.extract('(\d+\ \d*)',expand = False)
s2 = s2.str.replace(' ','')
s2 = s2.astype(float)
df_auto['Price BGN'] = s1 * s2 
df_auto.head()

,Model,Price,Year,Kilometers,Engine_type,Horsepower,Price BGN
0,Abarth 124,250 000 лв.,февруари 2020 г.,11111,Бензин,"регион Варна, гр. Варна11:26 часа от днес",250000.0
1,Abarth 595 595 PISTA 19000km,39 999 лв.,февруари 2020 г.,19000,Бензин,165,39999.0
2,Abarth 595,45 800 лв.,юни 2019 г.,54350,Бензин,181,45800.0
3,Abarth 595 Pista,36 000 лв.,септември 2017 г.,45000,Бензин,165,36000.0
4,Abarth 595,21 800 лв.,април 2014 г.,80000,Бензин,165,21800.0


In [10]:
del df_auto['Price']

In [11]:
df_auto = df_auto[['Model', 'Price BGN', 'Year', 'Kilometers', 'Engine_type', 'Horsepower']]
df_auto.head()

,Model,Price BGN,Year,Kilometers,Engine_type,Horsepower
0,Abarth 124,250000.0,февруари 2020 г.,11111,Бензин,"регион Варна, гр. Варна11:26 часа от днес"
1,Abarth 595 595 PISTA 19000km,39999.0,февруари 2020 г.,19000,Бензин,165
2,Abarth 595,45800.0,юни 2019 г.,54350,Бензин,181
3,Abarth 595 Pista,36000.0,септември 2017 г.,45000,Бензин,165
4,Abarth 595,21800.0,април 2014 г.,80000,Бензин,165


In [12]:
df_auto.rename(columns={'Year':'Year Manifacture', 'Engine_type':'Engine Type'}, inplace=True)
df_auto.head()

,Model,Price BGN,Year Manifacture,Kilometers,Engine Type,Horsepower
0,Abarth 124,250000.0,февруари 2020 г.,11111,Бензин,"регион Варна, гр. Варна11:26 часа от днес"
1,Abarth 595 595 PISTA 19000km,39999.0,февруари 2020 г.,19000,Бензин,165
2,Abarth 595,45800.0,юни 2019 г.,54350,Бензин,181
3,Abarth 595 Pista,36000.0,септември 2017 г.,45000,Бензин,165
4,Abarth 595,21800.0,април 2014 г.,80000,Бензин,165


In [13]:
# replace blank spaces with np.nan and clean Year Manifacture
df_auto = df_auto.replace(r'^\s*$', np.nan, regex=True)
df_auto = df_auto.dropna(subset = ["Year Manifacture"])
df_auto.head()

,Model,Price BGN,Year Manifacture,Kilometers,Engine Type,Horsepower
0,Abarth 124,250000.0,февруари 2020 г.,11111,Бензин,"регион Варна, гр. Варна11:26 часа от днес"
1,Abarth 595 595 PISTA 19000km,39999.0,февруари 2020 г.,19000,Бензин,165
2,Abarth 595,45800.0,юни 2019 г.,54350,Бензин,181
3,Abarth 595 Pista,36000.0,септември 2017 г.,45000,Бензин,165
4,Abarth 595,21800.0,април 2014 г.,80000,Бензин,165


In [14]:
df_auto['Year Manifacture'].unique()

array(['февруари 2020 г.', 'юни 2019 г.', 'септември 2017 г.',
       'април 2014 г.', 'май 2015 г.', 'септември 2010 г.',
       'август 2021 г.', 'декември 2017 г.', 'април 2009 г.',
       'септември 2014 г.', 'март 2008 г.', 'юни 2013 г.', 'юни 2005 г.',
       'септември 2006 г.', 'юни 2008 г.', 'май 2018 г.',
       'август 2015 г.', 'февруари 2012 г.', 'ноември 2008 г.',
       'декември 2008 г.', 'април 2019 г.', 'март 2012 г.',
       'февруари 2011 г.', 'август 2006 г.', 'март 2003 г.',
       'април 2010 г.', 'април 2015 г.', 'януари 2015 г.', 'юли 2015 г.',
       'октомври 2001 г.', 'април 2012 г.', 'февруари 2013 г.',
       'юли 2010 г.', 'март 2014 г.', 'октомври 2014 г.', 'април 2008 г.',
       'април 2002 г.', 'декември 2006 г.', 'януари 2019 г.',
       'май 2003 г.', 'септември 2011 г.', 'май 2002 г.',
       'февруари 2006 г.', 'декември 2013 г.', 'декември 2007 г.',
       'август 2008 г.', 'септември 2020 г.', 'октомври 2006 г.',
       'декември 2014 г.', 'януа

In [15]:
# extract years from "Year Manifacture" 
df_auto['Year Manifacture'] = df_auto['Year Manifacture'].str.extract(r'(\d+)',expand = False)

In [16]:
df_auto = df_auto[df_auto['Year Manifacture'].notna()]

In [17]:
df_auto.head()

,Model,Price BGN,Year Manifacture,Kilometers,Engine Type,Horsepower
0,Abarth 124,250000.0,2020,11111,Бензин,"регион Варна, гр. Варна11:26 часа от днес"
1,Abarth 595 595 PISTA 19000km,39999.0,2020,19000,Бензин,165
2,Abarth 595,45800.0,2019,54350,Бензин,181
3,Abarth 595 Pista,36000.0,2017,45000,Бензин,165
4,Abarth 595,21800.0,2014,80000,Бензин,165


In [18]:
df_auto["Year Manifacture_new"] = df_auto["Year Manifacture"].apply(lambda row: f'12/31/{row}')
df_auto.head()

,Model,Price BGN,Year Manifacture,Kilometers,Engine Type,Horsepower,Year Manifacture_new
0,Abarth 124,250000.0,2020,11111,Бензин,"регион Варна, гр. Варна11:26 часа от днес",12/31/2020
1,Abarth 595 595 PISTA 19000km,39999.0,2020,19000,Бензин,165,12/31/2020
2,Abarth 595,45800.0,2019,54350,Бензин,181,12/31/2019
3,Abarth 595 Pista,36000.0,2017,45000,Бензин,165,12/31/2017
4,Abarth 595,21800.0,2014,80000,Бензин,165,12/31/2014


In [19]:
# parce into DateTime
df_auto["Year Manifacture_new"] = [parser.parse(str(x)) for x in df_auto["Year Manifacture_new"]]
df_auto.head()

,Model,Price BGN,Year Manifacture,Kilometers,Engine Type,Horsepower,Year Manifacture_new
0,Abarth 124,250000.0,2020,11111,Бензин,"регион Варна, гр. Варна11:26 часа от днес",2020-12-31
1,Abarth 595 595 PISTA 19000km,39999.0,2020,19000,Бензин,165,2020-12-31
2,Abarth 595,45800.0,2019,54350,Бензин,181,2019-12-31
3,Abarth 595 Pista,36000.0,2017,45000,Бензин,165,2017-12-31
4,Abarth 595,21800.0,2014,80000,Бензин,165,2014-12-31


In [20]:
# extract "year" from Series
df_auto['Year Manifacture_t'] = df_auto['Year Manifacture_new'].dt.year
df_auto.head()

,Model,Price BGN,Year Manifacture,Kilometers,Engine Type,Horsepower,Year Manifacture_new,Year Manifacture_t
0,Abarth 124,250000.0,2020,11111,Бензин,"регион Варна, гр. Варна11:26 часа от днес",2020-12-31,2020
1,Abarth 595 595 PISTA 19000km,39999.0,2020,19000,Бензин,165,2020-12-31,2020
2,Abarth 595,45800.0,2019,54350,Бензин,181,2019-12-31,2019
3,Abarth 595 Pista,36000.0,2017,45000,Бензин,165,2017-12-31,2017
4,Abarth 595,21800.0,2014,80000,Бензин,165,2014-12-31,2014


In [21]:
del df_auto['Year Manifacture']
del df_auto['Year Manifacture_new']

In [22]:
df_auto.rename(columns={'Year Manifacture_t' : 'Year Manifacture'}, inplace=True)
df_auto.head()

,Model,Price BGN,Kilometers,Engine Type,Horsepower,Year Manifacture
0,Abarth 124,250000.0,11111,Бензин,"регион Варна, гр. Варна11:26 часа от днес",2020
1,Abarth 595 595 PISTA 19000km,39999.0,19000,Бензин,165,2020
2,Abarth 595,45800.0,54350,Бензин,181,2019
3,Abarth 595 Pista,36000.0,45000,Бензин,165,2017
4,Abarth 595,21800.0,80000,Бензин,165,2014


In [23]:
df_auto = df_auto[['Model', 'Price BGN', 'Year Manifacture', 'Kilometers', 'Engine Type', 'Horsepower']]
df_auto.head()

,Model,Price BGN,Year Manifacture,Kilometers,Engine Type,Horsepower
0,Abarth 124,250000.0,2020,11111,Бензин,"регион Варна, гр. Варна11:26 часа от днес"
1,Abarth 595 595 PISTA 19000km,39999.0,2020,19000,Бензин,165
2,Abarth 595,45800.0,2019,54350,Бензин,181
3,Abarth 595 Pista,36000.0,2017,45000,Бензин,165
4,Abarth 595,21800.0,2014,80000,Бензин,165


In [24]:
df_auto['Kilometers'].unique()

array(['11111', '19000', '54350', ..., '163145', '204865', '156353'],
      dtype=object)

In [25]:
df_auto['Kilometers'] = pd.to_numeric(df_auto['Kilometers'],errors = 'coerce')

In [26]:
df_auto['Horsepower'].unique()

array(['регион Варна, гр. Варна11:26 часа от днес', '165', '181', ...,
       'цвят Оранжев, Перфектен автомобил с дребни външни забележки. Нов...Особености - 4(5) Врати, Автоматичен контрол на стабилност... , регион София, гр. София12:10 часа от 10.12.2022',
       'цвят Сив, Добър за годините си автомобил. Има забележки по в...Особености - 4(5) Врати, Въздушни възглавници - Предни, Въ... , регион София, гр. София12:01 часа от 10.12.2022',
       'регион Плевен, гр. Кнежа11:02 часа от 10.12.2022'], dtype=object)

In [27]:
df_auto['Horsepower'] = pd.to_numeric(df_auto['Horsepower'],errors = 'coerce')
df_auto.head()

,Model,Price BGN,Year Manifacture,Kilometers,Engine Type,Horsepower
0,Abarth 124,250000.0,2020,11111,Бензин,NaN
1,Abarth 595 595 PISTA 19000km,39999.0,2020,19000,Бензин,165.0
2,Abarth 595,45800.0,2019,54350,Бензин,181.0
3,Abarth 595 Pista,36000.0,2017,45000,Бензин,165.0
4,Abarth 595,21800.0,2014,80000,Бензин,165.0


In [28]:
currentYear = datetime.now().year
df_auto['Years Automobile'] = currentYear - df_auto['Year Manifacture']

In [29]:
# rearrange column indexes
df_auto = df_auto[['Model', 'Year Manifacture','Years Automobile', 'Price BGN', 'Kilometers', 
                   'Engine Type', 'Horsepower']]
df_auto.head()

,Model,Year Manifacture,Years Automobile,Price BGN,Kilometers,Engine Type,Horsepower
0,Abarth 124,2020,2,250000.0,11111,Бензин,NaN
1,Abarth 595 595 PISTA 19000km,2020,2,39999.0,19000,Бензин,165.0
2,Abarth 595,2019,3,45800.0,54350,Бензин,181.0
3,Abarth 595 Pista,2017,5,36000.0,45000,Бензин,165.0
4,Abarth 595,2014,8,21800.0,80000,Бензин,165.0


# Filter and create DataFrames

In [30]:
df_auto =  df_auto.loc[(df_auto['Kilometers'] > 1111)]
df_auto =  df_auto.loc[(df_auto['Years Automobile'] > 0)]              
df_auto.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 38133 entries, 0 to 39282
Data columns (total 7 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   Model             38133 non-null  object 
 1   Year Manifacture  38133 non-null  int64  
 2   Years Automobile  38133 non-null  int64  
 3   Price BGN         37869 non-null  float64
 4   Kilometers        38133 non-null  int64  
 5   Engine Type       38133 non-null  object 
 6   Horsepower        36014 non-null  float64
dtypes: float64(2), int64(3), object(2)
memory usage: 2.3+ MB


In [31]:
df_auto["Year Manifacture"].unique()

array([2020, 2019, 2017, 2014, 2015, 2010, 2021, 2009, 2008, 2013, 2005,
       2006, 2018, 2012, 2011, 2003, 2001, 2002, 2007, 2016, 1999, 2004,
       1998, 1997, 2000, 1995, 1992, 1988, 1996, 1990, 1993, 1994, 1981,
       1986, 1991, 1977, 1966, 1960, 1985, 1975, 1969, 1958, 1987, 1983,
       1967, 1980, 1968, 1978, 1984, 1979, 1972, 1974, 1965, 1962, 1989,
       1973, 1953, 1970, 1971, 1963, 1964, 1930, 1982, 1959, 1976, 1934,
       1951, 1935], dtype=int64)

In [32]:
df_auto.head()

,Model,Year Manifacture,Years Automobile,Price BGN,Kilometers,Engine Type,Horsepower
0,Abarth 124,2020,2,250000.0,11111,Бензин,NaN
1,Abarth 595 595 PISTA 19000km,2020,2,39999.0,19000,Бензин,165.0
2,Abarth 595,2019,3,45800.0,54350,Бензин,181.0
3,Abarth 595 Pista,2017,5,36000.0,45000,Бензин,165.0
4,Abarth 595,2014,8,21800.0,80000,Бензин,165.0


In [33]:
df_auto['Engine Type'].unique()

array(['Бензин', 'Дизел', 'Хибриден', 'Електрически', 'Plug-in хибрид'],
      dtype=object)

In [36]:
filter1 = df_auto['Engine Type'].isin(['Дизел'])
df_diesel = df_auto[filter1]
df_diesel

,Model,Year Manifacture,Years Automobile,Price BGN,Kilometers,Engine Type,Horsepower
30,Alpina D3 Biturbo,2009,13,19000.0,203600,Дизел,214.0
31,Alpina XD3 biturbo,2015,7,74000.0,131000,Дизел,350.0
34,Audi A4 1.9,2001,21,4800.0,240000,Дизел,131.0
35,Audi A4 2.0TDI,2012,10,19650.0,132000,Дизел,120.0
37,Audi A4 3.0TDI Avant,2013,9,37500.0,309072,Дизел,245.0
...,...,...,...,...,...,...,...
39273,VW Passat 1.9tdi АВТОМАТ 131к.с.СЕДАН,2002,20,4999.0,234512,Дизел,131.0
39274,VW Passat Комби,2005,17,5800.0,270000,Дизел,131.0
39275,VW Golf 1.9 TDI,1999,23,3900.0,200000,Дизел,110.0
39277,VW Passat 2.0TDI-SPORT,2006,16,6999.0,163145,Дизел,140.0


In [37]:
filter2 = df_auto['Engine Type'].isin(['Бензин'])
df_gasoline = df_auto[filter2]
df_gasoline.head()

,Model,Year Manifacture,Years Automobile,Price BGN,Kilometers,Engine Type,Horsepower
0,Abarth 124,2020,2,250000.0,11111,Бензин,NaN
1,Abarth 595 595 PISTA 19000km,2020,2,39999.0,19000,Бензин,165.0
2,Abarth 595,2019,3,45800.0,54350,Бензин,181.0
3,Abarth 595 Pista,2017,5,36000.0,45000,Бензин,165.0
4,Abarth 595,2014,8,21800.0,80000,Бензин,165.0


In [38]:
filter3 = df_auto['Engine Type'].isin(['Електрически'])
df_electric = df_auto[filter3]
df_electric.head()

,Model,Year Manifacture,Years Automobile,Price BGN,Kilometers,Engine Type,Horsepower
292,Audi Q4 35/ E-Tron/ MATRIX/ PANORAMA/ NAVI/ LA...,2021,1,109980.0,12500,Електрически,170.0
296,Audi E-Tron SPORTBACK 55/ QUATTRO/ S-LINE/ 360...,2021,1,139980.0,23923,Електрически,408.0
522,Audi E-Tron Audi E-Tron Q4 50quattro sportback,2020,2,131700.0,13200,Електрически,299.0
2755,Chevrolet Volt,2017,5,46500.0,119000,Електрически,NaN
2839,Chevrolet Volt,2017,5,45000.0,89100,Електрически,150.0


In [39]:
filter4 = df_auto['Engine Type'].isin(['Хибриден', 'хибриден','Plug-in хибрид'])
df_hybrid = df_auto[filter4]
df_hybrid.head()

,Model,Year Manifacture,Years Automobile,Price BGN,Kilometers,Engine Type,Horsepower
146,Audi Q5 2.0 TFSI Hybrid Quattro tiptronic S-LINE,2012,10,31500.0,147000,Хибриден,211.0
172,Audi A6 HYBRID AUTOMATIC EURO 6D,2018,4,62500.0,223000,Хибриден,204.0
1212,Audi A3 E - Tron 1.4 Premium,2016,6,45800.0,61400,Plug-in хибрид,204.0
2118,Alfa Romeo 156 sportwagon Лъжец,2015,7,1000.0,393932,Хибриден,NaN
2505,Bentley Bentayga HYBRID/ FIRST EDITION/ NAIM/ ...,2021,1,440661.0,27630,Хибриден,449.0


In [40]:
currentDay = datetime.now().day
currentMonth = datetime.now().month
currentYear = datetime.now().year
e ='%s.%s.%s'%(currentDay,currentMonth,currentYear)
file_name = 'auto.bg'
writer = pd.ExcelWriter(f'{file_name}{e}.xlsx', engine='openpyxl')
df_diesel.to_excel(writer, sheet_name='Diesel engines')
df_gasoline.to_excel(writer, sheet_name='Gasoline engines')
df_electric.to_excel(writer, sheet_name='Electric engines')
df_hybrid.to_excel(writer, sheet_name='Hybrid engines')
writer.save()